In [1]:
pip install rdkit-pypi torch

  Using cached rdkit_pypi-2022.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.9 kB)
Using cached rdkit_pypi-2022.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (29.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [3]:
def smiles_to_fp(smiles, radius=2, n_bits=2048):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            # This generates a bit vector, which is a sequence of 1s and 0s.
            # It corresponds to the abscence or prescence of certain molecular features.
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
            return np.array(fp)
    except:
        return None
    return None

In [4]:
# Here we decide if the molecule is active or not
# Active = 1 if IC50/Ki/etc. < 1000 nM, otherwise inactive
def label_activity(row):
    if row['Activity Type'] in ['IC50', 'Ki', 'EC50']:
        try:
            value = float(row['Value'])
            return 1 if value < 1000 else 0
        except:
            return None
    return None

In [5]:
import os

# We make a dataframe from our compounds CSV.
df = pd.read_csv("alzheimers_bioactive_compounds.csv")

# We label the activity of each molecule (1 or 0).
df['Label'] = df.apply(label_activity, axis=1)
df = df.dropna(subset=['Label'])

# Generate fingerprints
df = df.dropna(subset=['SMILES'])
df['SMILES'] = df['SMILES'].astype(str)
df['Fingerprint'] = df['SMILES'].apply(smiles_to_fp)
df = df.dropna(subset=['Fingerprint'])

X = np.array(df['Fingerprint'].tolist())
y = df['Label'].astype(int).values

# We define a dictionary mapping for pathways and associated genes/compounds. 
# TODO: Add more mappings
pathway_targets = {
    "amyloid-beta": ["APP", "BACE1", "PSEN1", "APOE", "PLAU", "ABCA7", "SORL1", "SNCA", "PRNP", "A2M", "CSF1R"],
    "tau": ["MAPT", "GSK3B", "CDK5", "VCP", "PRNP", "A2M", "GRN"],
    "inflammation": ["TREM2", "TNF", "IL1B", "NLRP3", "APOE", "PLAU", "A2M", "GRN", "CSF1R"],
    "oxidative-stress": ["SOD1", "GPX1", "NFE2L2", "NOS1", "NOS2", "NOS3", "MT-ND1"]
}

# Sets the output labels if the target is found in the above mappings
def get_pathway_labels(target):
    labels = [0, 0, 0, 0]
    # keys = pathway_targets.keys()
    # print(keys)
    if target in pathway_targets["amyloid-beta"]:
        labels[0] = 1
    if target in pathway_targets["tau"]:
        labels[1] = 1
    if target in pathway_targets["inflammation"]:
        labels[2] = 1
    if target in pathway_targets["oxidative-stress"]:
        labels[3] = 1
    return labels
        
df["Pathway Labels"] = df["Gene Symbol"].apply(get_pathway_labels)
y = np.array(df["Pathway Labels"].to_list())

# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [6]:
class FingerprintDataset(Dataset):
    def __init__(self, X,Y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.Y = torch.tensor(Y, dtype=torch.float32)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

train_ds = FingerprintDataset(X_train, y_train)
val_ds = FingerprintDataset(X_val, y_val)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)

In [7]:
class SimpleNN(nn.Module):
    def __init__(self, input_dim, output_dim=4):
        super(SimpleNN, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 256), # Learn broad features
            nn.ReLU(),
            nn.Dropout(0.3), # 30% probability that randomly selected neurons are ignored
            nn.Linear(256, 64), # Learn more focused features
            nn.ReLU(),
            nn.Linear(64, output_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.fc(x)

In [8]:
def train(model, dataloader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    for xb, yb in dataloader:
        preds = model(xb)
        loss = loss_fn(preds, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

In [9]:
def evaluate(model, dataloader, loss_fn, threshold=0.5):
    """
    Evaluate the performance of a trained model on a dataset.

    This function computes the total loss and accuracy of the model's predictions
    compared to the truth labels provided in the dataloader.

    Parameters:
    - model: The trained PyTorch model
    - dataloader: A PyTorch DataLoader that provides batches of input data (xb) 
      and corresponding labels (yb).
    - loss_fn: A loss function to compute the model's performance (should use binary cross-entropy).
    - threshold: A float value used to convert model predictions to binary

    Returns:
    - A dictionary containing:
        - "loss": The average loss over all batches in the dataloader.
        - "accuracy_per_pathway": An array of accuracy values for each class/pathway.
        - "average_accuracy": A single value representing the overall accuracy across all classes.
    """
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for xb, yb in dataloader:
            preds = model(xb)
            loss = loss_fn(preds, yb)
            total_loss += loss.item()
            all_preds.append((preds > threshold).float())
            all_labels.append(yb)

    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    # Simple accuracy per pathway
    correct_per_class = (all_preds == all_labels).sum(dim=0)
    total_per_class = all_labels.shape[0]

    accuracy_per_class = (correct_per_class / total_per_class).numpy()
    avg_accuracy = (all_preds == all_labels).float().mean().item()

    return {
        "loss": total_loss / len(dataloader),
        "accuracy_per_pathway": accuracy_per_class,
        "average_accuracy": avg_accuracy
    }

In [10]:
# model = SimpleNN(input_dim=X_train.shape[1])
# loss_fn = nn.BCELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# def evaluate(loader):
#     model.eval()
#     correct, total = 0, 0
#     with torch.no_grad():
#         for xb, yb in loader:
#             preds = model(xb) > 0.5
#             correct += (preds.int() == yb.int()).sum().item()
#             total += yb.size(0)
#     return correct / total

# for epoch in range(20):
#     model.train()
#     for xb, yb in train_loader:
#         preds = model(xb)
#         loss = loss_fn(preds, yb)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
    
#     acc = evaluate(val_loader)
#     print(f"Epoch {epoch+1}: Val Accuracy = {acc:.4f}")

model = SimpleNN(input_dim=2048, output_dim=4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.BCELoss()

for epoch in range(1, 21):
    train_loss = train(model, train_loader, optimizer, loss_fn)
    val_metrics = evaluate(model, val_loader, loss_fn)

    print(f"Epoch {epoch:02d} | "
          f"Train Loss: {train_loss:.4f} | "
          f"Val Loss: {val_metrics['loss']:.4f} | "
          f"Avg Acc: {val_metrics['average_accuracy']:.4f}")

Epoch 01 | Train Loss: 0.2366 | Val Loss: 0.0616 | Avg Acc: 0.9790
Epoch 02 | Train Loss: 0.0224 | Val Loss: 0.0314 | Avg Acc: 0.9897
Epoch 03 | Train Loss: 0.0087 | Val Loss: 0.0280 | Avg Acc: 0.9911
Epoch 04 | Train Loss: 0.0051 | Val Loss: 0.0260 | Avg Acc: 0.9921
Epoch 05 | Train Loss: 0.0036 | Val Loss: 0.0263 | Avg Acc: 0.9897
Epoch 06 | Train Loss: 0.0028 | Val Loss: 0.0259 | Avg Acc: 0.9911
Epoch 07 | Train Loss: 0.0026 | Val Loss: 0.0262 | Avg Acc: 0.9907
Epoch 08 | Train Loss: 0.0030 | Val Loss: 0.0256 | Avg Acc: 0.9916
Epoch 09 | Train Loss: 0.0027 | Val Loss: 0.0295 | Avg Acc: 0.9902
Epoch 10 | Train Loss: 0.0026 | Val Loss: 0.0271 | Avg Acc: 0.9911
Epoch 11 | Train Loss: 0.0024 | Val Loss: 0.0280 | Avg Acc: 0.9907
Epoch 12 | Train Loss: 0.0027 | Val Loss: 0.0279 | Avg Acc: 0.9907
Epoch 13 | Train Loss: 0.0020 | Val Loss: 0.0277 | Avg Acc: 0.9907
Epoch 14 | Train Loss: 0.0020 | Val Loss: 0.0281 | Avg Acc: 0.9907
Epoch 15 | Train Loss: 0.0020 | Val Loss: 0.0280 | Avg Acc: 0.

In [11]:
def predict_smiles(smiles):
    """
    Parameters:
    - smiles: The SMILES code of the molecule you want to predict for

    Returns:
    - A string containing the data
    - A dictionary containing the data
    """
    output_str = f""
    output = {}
    
    fp = smiles_to_fp(smiles)
    if fp is None:
        return "", {}
    fp = scaler.transform([fp])
    pathways = ["amyloid-beta", "tau", "inflammation", "oxidative-stress"]
    with torch.no_grad():
        preds = model(torch.tensor(fp, dtype=torch.float32))
        for p in preds:
            for i, score in enumerate(p):
                output_str += f"{pathways[i]}: {score:.3f}"
                output_str += "\n"
                output[pathways[i]] = score
    return output_str, output
    # return preds

In [21]:
# predict_smiles("CCCN(Cc1cc(Cl)c1C#N)C1CCNC1")

# Read the CSV of naturally-occuring/environmetal compounds
natural_df = pd.read_csv("Natural Compounds Spreadhseet - Sheet1 (1).csv")

all_results = {
    "Name":[],
    "amyloid-beta":[],
    "tau":[],
    "inflammation":[],
    "oxidative-stress":[]
}
# Loop through each row and predict the SMILES
for index, row in natural_df.iterrows():
    output_str, output = predict_smiles(row["SMILES"])

    name = row["Compound Name"]
    all_results["Name"].append(name)
    
    for pathway in all_results.keys():
        if pathway != "Name":  # Skip adding the compound name
            prob = output.get(pathway, 0)  # Get the probability, default to 0 if not found
            val = prob if not torch.is_tensor(prob) else round(prob.item(), 3)
            all_results[pathway].append(val)

            # If the probability of activity against any pathway is greater than 50%, print it out
            if prob > 0.5:
                print(f"{name}:")
                print(output_str)
                print("")
                
results_df = pd.DataFrame(all_results)

# Write the DataFrame to a CSV file
results_df.to_csv('pathway_probabilities.csv', index=False)

# predict_smiles("CS(=O)CCCCN=C=S")

Curcumin:
amyloid-beta: 0.667
tau: 0.723
inflammation: 0.709
oxidative-stress: 0.017


Curcumin:
amyloid-beta: 0.667
tau: 0.723
inflammation: 0.709
oxidative-stress: 0.017


Curcumin:
amyloid-beta: 0.667
tau: 0.723
inflammation: 0.709
oxidative-stress: 0.017


Quercetin:
amyloid-beta: 0.032
tau: 0.000
inflammation: 0.986
oxidative-stress: 0.000


Caffeine:
amyloid-beta: 0.000
tau: 0.000
inflammation: 0.998
oxidative-stress: 0.001


Capsaicin:
amyloid-beta: 0.046
tau: 0.148
inflammation: 0.001
oxidative-stress: 0.552


Resveratrol:
amyloid-beta: 0.997
tau: 0.000
inflammation: 0.999
oxidative-stress: 0.000


Resveratrol:
amyloid-beta: 0.997
tau: 0.000
inflammation: 0.999
oxidative-stress: 0.000


Lycopene:
amyloid-beta: 0.012
tau: 0.001
inflammation: 0.000
oxidative-stress: 0.923


Epigallocatechin gallate:
amyloid-beta: 0.903
tau: 0.010
inflammation: 0.037
oxidative-stress: 0.000


Menthol:
amyloid-beta: 0.701
tau: 0.253
inflammation: 0.037
oxidative-stress: 0.189


Allicin:
amyloid-bet

[13:30:53] Explicit valence for atom # 12 O, 3, is greater than permitted


Withaferin A:
amyloid-beta: 0.566
tau: 0.126
inflammation: 0.624
oxidative-stress: 0.008


Withaferin A:
amyloid-beta: 0.566
tau: 0.126
inflammation: 0.624
oxidative-stress: 0.008


Baicalein:
amyloid-beta: 0.032
tau: 0.014
inflammation: 0.678
oxidative-stress: 0.002


Magnolol:
amyloid-beta: 0.002
tau: 0.000
inflammation: 0.048
oxidative-stress: 0.798


Uridine:
amyloid-beta: 0.987
tau: 0.265
inflammation: 0.953
oxidative-stress: 0.002


Uridine:
amyloid-beta: 0.987
tau: 0.265
inflammation: 0.953
oxidative-stress: 0.002


Punicic acid:
amyloid-beta: 0.001
tau: 0.001
inflammation: 0.001
oxidative-stress: 0.991


Gossypol:
amyloid-beta: 0.001
tau: 0.036
inflammation: 0.900
oxidative-stress: 0.070


Tetrahydrocannabinol (THC):
amyloid-beta: 0.004
tau: 0.031
inflammation: 0.042
oxidative-stress: 0.784


Eugenol:
amyloid-beta: 0.043
tau: 0.000
inflammation: 0.000
oxidative-stress: 0.693


Ajmaline:
amyloid-beta: 0.953
tau: 0.582
inflammation: 0.419
oxidative-stress: 0.011


Ajmaline:
amylo

[13:30:53] SMILES Parse Error: unclosed ring for input: 'CC1=C2[C@@H]3CN4CC[C@@]5(C[C@@H]3N2CC6=C1C=CC=C6OC)C4C5=CNC7=C6C=C(C=C7)OC'


Lapachol:
amyloid-beta: 0.036
tau: 0.002
inflammation: 0.988
oxidative-stress: 0.024


Tylophorine:
amyloid-beta: 0.005
tau: 0.012
inflammation: 0.005
oxidative-stress: 0.966


Esculetin:
amyloid-beta: 0.911
tau: 0.049
inflammation: 0.890
oxidative-stress: 0.038


Esculetin:
amyloid-beta: 0.911
tau: 0.049
inflammation: 0.890
oxidative-stress: 0.038


Lupeol:
amyloid-beta: 0.139
tau: 0.490
inflammation: 0.507
oxidative-stress: 0.174


Xanthohumol:
amyloid-beta: 0.951
tau: 0.005
inflammation: 0.920
oxidative-stress: 0.005


Xanthohumol:
amyloid-beta: 0.951
tau: 0.005
inflammation: 0.920
oxidative-stress: 0.005


Morin:
amyloid-beta: 0.010
tau: 0.000
inflammation: 0.997
oxidative-stress: 0.003


Fisetin:
amyloid-beta: 0.032
tau: 0.014
inflammation: 0.678
oxidative-stress: 0.002


Isorhamnetin:
amyloid-beta: 0.073
tau: 0.034
inflammation: 0.815
oxidative-stress: 0.010


Syringic acid:
amyloid-beta: 0.009
tau: 0.000
inflammation: 1.000
oxidative-stress: 0.000


Acteoside:
amyloid-beta: 0.99

In [13]:
# import random

# def generate_new_mol(seed_smiles, mutation_rate=0.1):
#     mol = Chem.MolFromSmiles(seed_smiles)

#     if mol == None:
#         print("here")
#         return seed_smiles
    
#     if random.random() < mutation_rate:
#         atom_idx = random.randint(0, mol.GetNumAtoms() - 1)
#         atom = mol.GetAtomWithIdx(atom_idx)
#         current_atomic_num = atom.GetAtomicNum()
        
#         # Define potential new atoms considering basic chemistry rules
#         potential_atoms = {
#             1: [6, 7, 8],  # H (1) can change to C (6), N (7), O (8)
#             6: [1, 7, 8, 9, 15],  # C (6) can change to H (1), N (7), O (8), F (9), P (15)
#             7: [6, 1, 8],  # N (7) can change to C (6), H (1), O (8)
#             8: [6, 7],  # O (8) can change to C (6), N (7)
#             # TODO: extend this mapping for more atoms based on bonding rules
#         }

#         # Select a new atomic number ensuring it’s a suitable mutation
#         new_atomic_num = current_atomic_num
#         if current_atomic_num in potential_atoms:
#             new_atomic_num = random.choice(potential_atoms[current_atomic_num])
#             atom.SetAtomicNum(new_atomic_num)

#         # Convert back to SMILES
#         new_smiles = Chem.MolToSmiles(mol)
#         return new_smiles
        
#     return seed_smiles  # Return original if no mutation occurs

In [14]:
# seed_smiles = "CC(C)(C)OC(=O)N1CCCC1CNC1CCC(c2cc(F)ccc2F)(S(=O)(=O)c2ccc(Cl)cc2)CC1"
# score = 0

# num_mutations = 0
# max_mutations = 20

# while num_mutations < max_mutations:
#     new = generate_new_mol(seed_smiles)
#     seed_smiles = new
#     print(seed_smiles)
#     # prob = predict_smiles(seed_smiles)
#     # if prob != None:
#     #     score = prob
#     #     print(score)
#     num_mutations += 1